In [ ]:
import numpy as np
from module import *
import pypoman
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from statsmodels.distributions.empirical_distribution import ECDF
import os
from plotting import make_rectangle

In [ ]:
np.random.seed(42)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
collapses = 0

blue_red_s = []
green_red_s = []
green_blue_s = []

parameters_beta = [np.array([1.0, 1.0]), np.array([0.5, 0.5])]

for parameter_beta in parameters_beta:
    blue_red = []
    green_red = []
    green_blue = []
    for i in range(1000):
        theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11 = generate_instance(beta = True, parameters_beta = parameter_beta)

        # Get the vectors alpha, beta with the marginal distributions P_{XZ} and P_{YZ}
        alpha = get_alpha(theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11)
        beta = get_beta(theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11)

        # Get the constraint matrices from the marginal probabilities
        A, B = get_constraint_matrices(theta_x, theta_y)

        # Get "trivial" marginal bounds---lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max
        lambda_x_min, lambda_x_max, a_0 = get_trivial_bounds(alpha)
        lambda_y_min, lambda_y_max, b_0 = get_trivial_bounds(beta)

        # Get the bounds from structural causal marginal problem through linear program
        LB_X, UB_X, LB_Y, UB_Y = get_linear_prog_bounds(a_0, b_0, A, B, lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max)

        tolerance = 1e-6
        collapse = False
        if np.abs(UB_X - LB_X)<tolerance or np.abs(UB_Y - LB_Y)<tolerance:
            collapse = True
            collapses+=1

        proj = compute_affine_projection(A, B, a_0, b_0)
        eq = compute_equality_params(A, B, a_0, b_0)
        ineq = compute_inequality_params(A, B, lambda_y_min, lambda_y_max, b_0, lambda_x_min, lambda_x_max, a_0)

        area_polygon = 0
        if collapse==False:
            # List of vertices of the projection of the polytope in R^{16}
            vertices_projected_polytope = pypoman.projection.project_polytope(proj=proj, 
                                                                              ineq=ineq, 
                                                                              eq=eq)

            points = np.vstack(vertices_projected_polytope)
            area_polygon = area_polygon_points(points)

        trivial_bounds = make_rectangle(lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max)
        area_trivial = area_polygon_points(trivial_bounds)
        lin_prog_bounds = make_rectangle(LB_X, UB_X, LB_Y, UB_Y)

        area_lin_prog = area_polygon_points(lin_prog_bounds)

        green_red.append(area_polygon/area_trivial)
        blue_red.append(area_lin_prog/area_trivial)
        green_blue.append(area_polygon/area_lin_prog)

    green_red_s.append(np.array(green_red))
    blue_red_s.append(np.array(blue_red))
    green_blue_s.append(np.array(green_blue))

In [ ]:
# figure_path = ''
# filename = os.path.join(figure_path, 'figure_3_c.pdf')

In [ ]:
linestyles = ['solid', 'dashed','dotted', 'dashdot']

plt.figure(figsize=(5,4))
for i in range(len(green_red_s)):
    additional_label = r'$\alpha=\beta=$'+str(parameters_beta[i][0])
    
    count_blue, bins_count_blue = np.histogram(blue_red_s[i][blue_red_s[i]<1.0], bins=10)
    pdf_blue = count_blue / sum(count_blue)
    cdf_blue = np.cumsum(pdf_blue)
    plt.plot(bins_count_blue[1:], cdf_blue, label=additional_label, color='#016ddb', alpha=0.9, linestyle=linestyles[i*2], linewidth=2.5)

    count_green, bins_count_green = np.histogram(green_red_s[i][green_red_s[i]<1.0], bins=10)
    pdf_green = count_green / sum(count_green)
    cdf_green = np.cumsum(pdf_green)
    plt.plot(bins_count_green[1:], cdf_green, label=additional_label, color='#009292', alpha=0.6, linestyle=linestyles[i*2+1], linewidth=2.5)

plt.legend(fontsize=13)
xlabel = 'Ratio with red area'
ylabel = 'CDF'
plt.xlabel(xlabel, fontsize=15, labelpad=5)
plt.ylabel(ylabel, fontsize=15, labelpad=5)
# plt.savefig(filename, dpi=None, facecolor='w', edgecolor='w',
#         orientation='portrait', format=None,
#         transparent=True, bbox_inches='tight', pad_inches=0.1, metadata=None)
plt.show()